### Project 5: Relational Database
### Author: Chad Bell
### Date: 3-10-2023
### Description: Recreate the PREVAIL database

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import psycopg2

In [13]:
# Establish a connection with the postgres database
con = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="example_password")
# Create a cursor object
cur = con.cursor()

In [28]:
# con.commit()
con.rollback()

In [ ]:
# Create a table
cur.execute("CREATE TABLE Loans (id serial PRIMARY KEY, loan_amount integer, initial_loan_start_date DATE, start_date DATE, end_date DATE);")

In [31]:
cur.execute("DROP TABLE IF EXISTS Clients;")
cur.execute("CREATE TABLE Clients (id serial PRIMARY KEY, first_name varchar(255) NOT NULL, last_name varchar(255), email varchar(255));")

In [32]:
cur.execute("SELECT * FROM information_schema.tables WHERE table_schema='public';")
cur.fetchall()

[('postgres',
  'public',
  'loans',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'clients',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None)]